In [1]:
from os import path
import numpy as np
import pandas as pd
from IPython.display import display

if not path.exists("data/raw/StreamingHistory.csv"):
    streaming_history_0_df = pd.read_json("data/raw/StreamingHistory0.json")
    streaming_history_1_df = pd.read_json("data/raw/StreamingHistory1.json")
    streaming_history_2_df = pd.read_json("data/raw/StreamingHistory2.json")
    streaming_history_3_df = pd.read_json("data/raw/StreamingHistory3.json")

    stream_history_df = pd.concat([streaming_history_0_df, streaming_history_1_df, streaming_history_2_df, streaming_history_3_df])
    stream_history_df.to_csv("data/raw/StreamingHistory.csv")
else:
    stream_history_df = pd.read_csv("data/raw/StreamingHistory.csv")
    stream_history_df.drop(stream_history_df.columns[0], axis=1, inplace=True)

print(stream_history_df.shape)
stream_history_df.head()

(32408, 4)


/home/jcroderman/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,endTime,artistName,trackName,msPlayed
0,2019-10-30 00:15,Greensky Bluegrass,Time > Breathe Reprise >,129948
1,2019-10-30 00:34,Greensky Bluegrass,Time > Breathe Reprise >,1360
2,2019-10-30 00:52,Matthew Barber,Sleep Please Come to Me,172520
3,2019-10-30 00:58,Matthew Barber,Somebody Sometime,262760
4,2019-10-30 00:58,Matthew Barber,Our Voices,30951


## Cleaning Up the Data

First, fix the date data

In [2]:
from datetime import timedelta

stream_history_df["date"] = pd.to_datetime(stream_history_df["endTime"], format="%Y-%m-%d")
stream_history_df["date"] = stream_history_df["date"].apply(lambda date: date - timedelta(hours = 6)) # Put it in CST
stream_history_df.drop("endTime", axis=1, inplace=True)
stream_history_df["date"].describe(datetime_is_numeric=True)

count                            32408
mean     2020-06-11 04:46:03.971241728
min                2019-10-29 18:15:00
25%                2020-02-17 22:24:15
50%                2020-06-17 23:21:00
75%                2020-10-06 15:26:00
max                2021-01-07 17:29:00
Name: date, dtype: object

In [3]:
stream_history_df = stream_history_df.rename({
    "artistName": "artist_name",
    "trackName": "track_name",
    "msPlayed": "ms_played"
}, axis=1)

Just for simplicity sake, I'm only going to focus on music listened to in 2020

In [4]:
stream_history_df = stream_history_df[~(stream_history_df['date'] < '2020-01-01') & ~(stream_history_df['date'] >= '2021-01-01')].reset_index(drop=True)
stream_history_df.date.describe(datetime_is_numeric=True)

count                            26837
mean     2020-07-11 04:24:29.672467200
min                2020-01-01 00:51:00
25%                2020-04-08 08:30:00
50%                2020-07-21 15:32:00
75%                2020-10-13 08:22:00
max                2020-12-31 23:59:00
Name: date, dtype: object

Before I get rid of anything, let's take a look at how long I listened to music or podcasts this year!

In [5]:
time_streamed_ms = sum(stream_history_df.ms_played)
time_streamed_sec = time_streamed_ms / 1000
time_streamed_min = time_streamed_sec / 60
time_streamed_hour = time_streamed_min / 60
time_streamed_day = time_streamed_hour / 24
print("Days I have listened to Spotify: {:.3f} days".format(time_streamed_day))

Days I have listened to Spotify: 54.349 days


## Consolidating Data

We're about to make a helluva lot of calls to Spotify, so we'll want to consolidate the data to make as few calls as possible. To do this, we'll gather all nonunique tracks

In [6]:
stream_history_consolidated_df = stream_history_df[["track_name", "artist_name"]].drop_duplicates().reset_index(drop=True)
print(stream_history_consolidated_df.shape)
stream_history_consolidated_df.head()

(5896, 2)


,track_name,artist_name
0,Haitian Fight Song,Mingus Big Band
1,Everything You Is,Andy Martin & Vic Lewis
2,To You,Thad Jones
3,Auld Lang Syne,Guy Lombardo & His Royal Canadians
4,All Of Me,Count Basie


Now, it's imperative that I get the IDs for all of these tracks, because if I have the IDs I'll be able to do all sorts of stuff including gathering additional data on the tracks. So here's my algorithm for extracting the ID from the `track_name` and `artist_name`:

1. Search for the track by calling Spotify's search API with the query `'track:"{track_name}" AND artist:"{artist_name}"'`. I'll only be searching for tracks, regardless of whether or not the thing being searched is a song or podcast episode
2. Take the list of all potential matches, and filter them. Only accept the track if the track name from the API call is exactly the same as the given `track_name`, and if the `artist_name` is included in the artist names of the track from the API call
3. If that list is nonempty, return the first result with type `track` (Note: this would be a good area to add an "accurate" option to the data. If the list is more than one element, have the user manually select which is correct [AKA throw error `Multiple matches found`])
4. If the track list is empty, then it is probably a podcast episode
5. Search for the episode by calling Spotify's search API with the query `'"{track_name}" AND "{artist_name}"'`, and only search for episodes
6. Filter the list of potential matches by accepting only if the name of the episode returned exactly matches `name`
7. If that list is empty, return `None`
8. If that list has 1 element, return that element's id with type `episode`
9. If that list has more than 1 element, we have to get the returned episodes' shows. To do that, we'll take the episode ids returned from Spotify's search endpoint and send it to Spotify's episode endpoint
10. Take the results from that and filter if the returned show name is the same as `show_name`
11. If that list is empty, return `None`
12. If that list is nonempty, return the first result with type `episode` (Note: same comment as above with the "accurate" option)

In [7]:
import requests
import re
import time
from urllib.parse import urlencode
token = open('token.txt', 'r').read().strip()

base_url = "https://api.spotify.com/v1/"
query_obj = {"type": "track,episode", "market": "US", "limit": 5}

def get_correct_episode(name, show_name):
    # We have a separate search call here without the `track:` and `artist:` search fields. That way we can search for podcasts
    query_obj["q"] = str.format('"{}" AND "{}"', re.sub(r"['\"]", "", name), re.sub(r"['\"]", "", show_name))
    res = requests.get(base_url + "search" + "?" + urlencode(query_obj), headers={
        "Authorization": "Bearer " + token
    })
    if res.status_code != 200:
        return pd.Series([None, None, "Search call failed, try again"])
    potential_episodes = res.json()["episodes"]["items"]

    episodes = list(filter(lambda episode: episode["name"] == name, potential_episodes))
    if len(episodes) == 0:
        return None
    elif len(episodes) == 1:
        return ["episode", episodes[0]["id"]]

    # The episode object returned does not have show data, so let's get that show data
    episode_ids = list(map(lambda episode: episode["id"], episodes))
    res = requests.get(base_url + "episodes" + "?" + urlencode({"ids": ",".join(episode_ids)}), headers={
        "Authorization": "Bearer " + token
    })
    if res.status_code != 200:
        return "Episodes call failed, try again"
    res_json = res.json()
    episodes = list(filter(lambda episode: episode["show"]["name"] == show_name, res_json["episodes"]))
    if len(episodes) > 0:
        return ["episode", episodes[0]["id"]]

    # Still nothing? Just return None then
    return None

def get_correct_track_or_episode(name, artist, potential_tracks):
    # First check for tracks
    tracks = list(filter(lambda track: track["name"] == name and
                                       any(list(map(lambda track_artist_obj: track_artist_obj["name"] == artist, track["artists"])))
                         , potential_tracks))
#     print(list(map(lambda track: track["name"], potential_tracks)))
    if len(tracks) > 0:
        return ["track", tracks[0]["id"]]

    # If no tracks follow the criteria, it may be a podcast episode
    return get_correct_episode(name, artist)

def get_spotify_data(track_name, artist_name):
    # Adding the `track:` and `artist:` search fields here disallow us from finding podcasts, only tracks
    query_obj["q"] = str.format('track:"{}" AND artist:"{}"', re.sub(r"['\"]", "", track_name), re.sub(r"['\"]", "", artist_name))
    res = requests.get(base_url + "search" + "?" + urlencode(query_obj), headers={
        "Authorization": "Bearer " + token
    })
    if res.status_code != 200:
        return pd.Series([None, None, "Track search call failed, try again"])
    res_json = res.json()

    data = get_correct_track_or_episode(track_name, artist_name, res_json["tracks"]["items"])
    if type(data) == str:
        return pd.Series([None, None, data])
    elif data == None:
        return pd.Series([None, None, "No data available"])
    else:
        return pd.Series([data[0], data[1], None])

Split the entries into buckets in case one fails we don't have to completely restart

In [8]:
import time
from functools import reduce

num_entries = stream_history_consolidated_df.shape[0]
bucket_size = 1000
num_buckets = num_entries // bucket_size + (1 if num_entries % bucket_size > 0 else 0)

bucket_times = [0] * num_buckets
stream_history_bucket_df = pd.DataFrame()
for i in range(num_buckets):
    if not path.exists("data/processed/history/StreamingHistoryWithData{}.csv".format(i)):
        time_start = time.time()
        stream_history_bucket_df = stream_history_consolidated_df.loc[i * bucket_size:((i + 1) * bucket_size) - 1].copy()
        stream_history_bucket_df[["type", "id", "error"]] = stream_history_bucket_df.apply(lambda x: get_spotify_data(x[0], x[1]), axis=1)
        time_elapsed = time.time() - time_start
        bucket_times[i] = time_elapsed
        print("Time elapsed for bucket {}: {:.3f} seconds".format(i, time_elapsed))
        stream_history_bucket_df.to_csv("data/processed/history/StreamingHistoryWithData{}.csv".format(i))

if stream_history_bucket_df.shape[0] == 0:
    print("Already collected all data")
else:
    # Last time I ran this it took 1944.531 seconds (32.409 minutes)
    total_time_elapsed = reduce(lambda acc, time_elapsed: acc + time_elapsed, bucket_times, 0)
    print("Total time elapsed: {:.3f} seconds ({:.3f} minutes)".format(total_time_elapsed, total_time_elapsed / 60))
    display(stream_history_bucket_df.tail())

Already collected all data


In [9]:
if not path.exists("data/processed/StreamingHistoryWithData.csv"):
    stream_history_consolidated_df = pd.DataFrame()
    for i in range(num_buckets):
        temp_df = pd.read_csv("data/processed/history/StreamingHistoryWithData{}.csv".format(i))
        temp_df.drop(temp_df.columns[0], axis=1, inplace=True)
        if stream_history_consolidated_df.shape[0] == 0:
            stream_history_consolidated_df = temp_df
        else:
            stream_history_consolidated_df = stream_history_consolidated_df.append(temp_df)

    stream_history_consolidated_df = stream_history_consolidated_df.reset_index(drop=True)
    stream_history_consolidated_df.to_csv("data/processed/StreamingHistoryWithData.csv")
else:
    stream_history_consolidated_df = pd.read_csv("data/processed/StreamingHistoryWithData.csv")
    stream_history_consolidated_df.drop(stream_history_consolidated_df.columns[0], axis=1, inplace=True)
stream_history_consolidated_df.tail()

,track_name,artist_name,type,id,error
5891,"Monday, December 28, 2020",Up First,NaN,NaN,No data available
5892,99: The 2020 Holiday Special! - The Gus & Eddy...,The Gus & Eddy Podcast,episode,0Fmh7UkdQ3GRzC8Vm3A3RA,NaN
5893,80: The Whistleblower,Darknet Diaries,episode,5UbRqOalKUzXjpT9uJaqF7,NaN
5894,Runnin' Red,Mandolin Orange,track,3qTSFH4kjCaVTEaGtF2LYZ,NaN
5895,Making Republicans Environmentalists Again,How to Save a Planet,episode,0H3vp2oz1tV3svXi2suaaZ,NaN


### Cleaning the Data
There are going to be a few entries that were missed by my algorithm. We'll take those and export them, so they can be manually fixed using the program I made for this, the [Spotify Data Fix Program](https://justinro-underscore.github.io/SpotifyDataFix/index.html)

In [10]:
missing_data_df = stream_history_consolidated_df[stream_history_consolidated_df["id"].isna()].reset_index()
print(missing_data_df.shape[0])
if not path.exists("data/processed/MissingStreamingHistoryWithId.csv"):
    missing_data_df.to_csv("data/processed/MissingStreamingHistoryWithId.csv")
missing_data_df.head()

99


,index,track_name,artist_name,type,id,error
0,91,Say Something,Guitar Tribute Players,NaN,NaN,No data available
1,267,Texas,George Strait,NaN,NaN,No data available
2,272,Don't Stop Me Now - 2011 Mix,Queen,NaN,NaN,No data available
3,536,When A Cowboy Trades His Spurs For Wings,Willie Watson,NaN,NaN,No data available
4,635,A Night In Tunisia (feat. Rochester Philharmon...,Dizzy Gillespie,NaN,NaN,No data available


Now go! Fix the data! Return when it's done!

In [11]:
if path.exists("data/processed/FixedDataForJustin.csv"):
    fixed_data_df = pd.read_csv("data/processed/FixedDataForJustin.csv")
    fixed_data_df.drop(fixed_data_df.columns[0], axis=1, inplace=True)
    fixed_data_df.index = fixed_data_df["index"]
    fixed_data_df.drop(columns=["index"], axis=1, inplace=True)
    display(fixed_data_df.head())
else:
    print("Please run the Spotify Fix")

,track_name,artist_name,type,id,delete
index,,,,,
91,Say Something,Guitar Tribute Players,track,0sV6V3wVIqWMlJ9PMJA4aE,False
267,Texas,George Strait,track,6PfNhcTuMItHBwVMaLTF9B,False
272,Don't Stop Me Now - 2011 Mix,Queen,track,7hQJA50XrCWABAu5v6QZ4i,False
536,When A Cowboy Trades His Spurs For Wings,Willie Watson,track,3NMDFORx33jILYPnSP4Jcg,False
635,A Night In Tunisia (feat. Rochester Philharmon...,Dizzy Gillespie,track,7j7MtQLO2Od1RQLu1yRpAZ,False


Let's go ahead and flag all of the tracks to delete

In [12]:
stream_history_consolidated_df["delete"] = False
to_delete_df = fixed_data_df[fixed_data_df["delete"]]
stream_history_consolidated_df.loc[to_delete_df.index, "delete"] = True
display(stream_history_consolidated_df.head())
stream_history_consolidated_df.iloc[to_delete_df.index].head()

,track_name,artist_name,type,id,error,delete
0,Haitian Fight Song,Mingus Big Band,track,4hns23kYYZg0BhDwXeDxB1,NaN,False
1,Everything You Is,Andy Martin & Vic Lewis,track,3sLmks6fCY40bBSGDjU4FO,NaN,False
2,To You,Thad Jones,track,2408a07TNDga6lMlaIFLEU,NaN,False
3,Auld Lang Syne,Guy Lombardo & His Royal Canadians,track,4Saza06xljloZwotqXdNle,NaN,False
4,All Of Me,Count Basie,track,0rupt7DuLo3WGecL3cyi19,NaN,False


,track_name,artist_name,type,id,error,delete
665,Find a Way,Matt Quentin,NaN,NaN,No data available,True
1974,Your new Spotify Home,What's New With Spotify,NaN,NaN,No data available,True
1999,Malicious Life Presents: ToTok,Darknet Diaries,NaN,NaN,No data available,True
2044,Announcement! We’re taking calls and broadcast...,Reply All,NaN,NaN,No data available,True
3307,Presenting: The Jordan Harbinger Show featurin...,Darknet Diaries,NaN,NaN,No data available,True


Now we can set the `type` and `id` of all the missing tracks

In [13]:
fixed_data_df = fixed_data_df[~fixed_data_df["delete"]]

stream_history_consolidated_df.loc[fixed_data_df.index, "type"] = fixed_data_df["type"]
stream_history_consolidated_df.loc[fixed_data_df.index, "id"] = fixed_data_df["id"]
stream_history_consolidated_df.iloc[fixed_data_df.index].head()

,track_name,artist_name,type,id,error,delete
91,Say Something,Guitar Tribute Players,track,0sV6V3wVIqWMlJ9PMJA4aE,No data available,False
267,Texas,George Strait,track,6PfNhcTuMItHBwVMaLTF9B,No data available,False
272,Don't Stop Me Now - 2011 Mix,Queen,track,7hQJA50XrCWABAu5v6QZ4i,No data available,False
536,When A Cowboy Trades His Spurs For Wings,Willie Watson,track,3NMDFORx33jILYPnSP4Jcg,No data available,False
635,A Night In Tunisia (feat. Rochester Philharmon...,Dizzy Gillespie,track,7j7MtQLO2Od1RQLu1yRpAZ,No data available,False


Not sure why I added this `error` column, didn't actually end up using it for anything:

In [14]:
stream_history_consolidated_df.drop(columns=["error"], axis=1, inplace=True)

Finally we can merge the two dataframes and remove the ones that need deleting. Now we have the full streaming history data!

In [15]:
stream_history_df = stream_history_df.merge(stream_history_consolidated_df, on=["artist_name", "track_name"])
stream_history_df = stream_history_df[~stream_history_df["delete"]].drop(columns=["delete"], axis=1)
stream_history_df = stream_history_df.sort_values(by=["date"]).reset_index(drop=True)
stream_history_df.head()

,artist_name,track_name,ms_played,date,type,id
0,Mingus Big Band,Haitian Fight Song,680,2020-01-01 00:51:00,track,4hns23kYYZg0BhDwXeDxB1
1,Andy Martin & Vic Lewis,Everything You Is,3282,2020-01-01 00:52:00,track,3sLmks6fCY40bBSGDjU4FO
2,Thad Jones,To You,1496,2020-01-01 00:52:00,track,2408a07TNDga6lMlaIFLEU
3,Guy Lombardo & His Royal Canadians,Auld Lang Syne,66171,2020-01-01 00:52:00,track,4Saza06xljloZwotqXdNle
4,Count Basie,All Of Me,150773,2020-01-01 00:55:00,track,0rupt7DuLo3WGecL3cyi19


In [16]:
stream_history_consolidated_df = stream_history_consolidated_df[~stream_history_consolidated_df["delete"]]
stream_history_consolidated_df.drop(columns=["delete"], axis=1, inplace=True)
stream_history_consolidated_df.head()

,track_name,artist_name,type,id
0,Haitian Fight Song,Mingus Big Band,track,4hns23kYYZg0BhDwXeDxB1
1,Everything You Is,Andy Martin & Vic Lewis,track,3sLmks6fCY40bBSGDjU4FO
2,To You,Thad Jones,track,2408a07TNDga6lMlaIFLEU
3,Auld Lang Syne,Guy Lombardo & His Royal Canadians,track,4Saza06xljloZwotqXdNle
4,All Of Me,Count Basie,track,0rupt7DuLo3WGecL3cyi19


Let's save this data:

In [17]:
if not path.exists("data/processed/FullTrackData.csv"):
    stream_history_consolidated_df.to_csv("data/processed/FullTrackData.csv")
if not path.exists("data/ready/CompleteStreamingHistory.csv"):
    stream_history_df.to_csv("data/ready/CompleteStreamingHistory.csv")

## Complete!

Now we can move along to the fun stuff... the ANALYSIS

For the future: I should probably finish cleaning the Podcast data as well